In [1]:
using StatsBase
using DelimitedFiles
import Base.string

mutable struct ge
    hp::Int16
    elf::Bool
end    

mutable struct tile
    empty::Bool
    char::Union{ge, Nothing}
end

ischar(t::tile) = t.char != nothing
istile(t::tile) = t.empty
isunoccupied(t::tile) = t.empty && !iself(t) && !isg(t)

iself(t::tile) = ischar(t) && t.char.elf
isg(t::tile) = ischar(t) && !t.char.elf

attack(t::tile) = begin
    t.char.hp -= 3
    #println("attack: hp left $(t.char.hp)")
    if t.char.hp <= 0
        return tile(true, nothing)
    else
        return t
    end
end

string(t::tile) = ischar(t) ? (iself(t) ? "E" : "G") : (isunoccupied(t) ? "." : "#")

function find_nearest_paths(pos_of_char, map, hw)    
    # initialise the variables
    found_nearest_opponent = false
    
    # vector of paths
    old_paths = Vector{CartesianIndex{2}}[]
    new_paths = Vector{CartesianIndex{2}}[]

    push!(old_paths, [pos_of_char])
    
    # the path map which contains shortest distance to various points
    path_map = Array{Int, 2}(undef, hw, hw)
    path_map .= hw*hw

    # dist keeps track of the distance from starting point
    dist = 0
    for op in old_paths
        path_map[op[end]] = dist
    end 
    
    # am I looking for an Elf or a Goblin?
    find_elf = isg(map[pos_of_char]) ? true : false

    # which element of old_paths am I up to?
    paths_upto = 1
    
    
    loop = 0
    # scan through the old_paths    
    #while (loop <= 2) && true
    while true
#         if mod(loop, 20000) == 0
#             writedlm("ok_$(loop).csv", path_map, ',')
#         end
        loop += 1
        if paths_upto > length(old_paths)
            if found_nearest_opponent
                #writedlm("ok_done_$(loop).csv", path_map, ',')
                return (new_paths, path_map, found_nearest_opponent)
            elseif length(new_paths) == 0
                #writedlm("ok_exhausted_$(loop).csv", path_map, ',')
                return (old_paths, path_map, found_nearest_opponent)
            end
            old_paths = new_paths
            new_paths = Vector{CartesianIndex{2}}[]
            paths_upto = 1

            dist += 1
        end
        
        # the starting position
        #println(old_paths[paths_upto])
        # obtain the last position in the path
        pos = old_paths[paths_upto][end]
        
        # the reachable position from pos
        #np = next_pos(pos, map, !iself(map[pos]))
        
        # the list of new positions to consider - 4 directions in total
        new_pos = CartesianIndex.([(pos[1], pos[2]-1), 
                    (pos[1]-1, pos[2]), 
                    (pos[1]+1, pos[2]),
                    (pos[1], pos[2]+1)])
        
        # if any of the new_pos contains the enemy you are looking for then can disregard the other paths        
        if find_elf
            # if any of the elements is an elf then we've found it!
            iself_vec = [iself(map[new_pos]) for new_pos in new_pos]            
            
            if any(iself_vec)
                # if I find a path then get rid of all paths that didn't lead to an Elf
                new_paths = filter(x->iself(map[x[end]]), new_paths)
                    
                    
                found_nearest_opponent = true
                # keep only those that have elf
                new_pos = new_pos[iself_vec]
                
                #println("woohoo found elf")
                #writedlm("ok_found_elf_$(loop).csv", path_map, ',')
            end
        else 
            # if any of the elements is an Goblin then we've found it!
            isg_vec = [isg(map[new_pos]) for new_pos in new_pos]
            
            if any(isg_vec) 
                new_paths = filter(x->isg(map[x[end]]), new_paths)                        
                found_nearest_opponent = true
                # keep only those that have elf
                new_pos = new_pos[isg_vec]
                #println("woohoo found Goblin")
                #writedlm("ok_found_goblin_$(loop).csv", path_map, ',')                
            end
        end
        
        # if not found the nearest enemy then remove the walls and blocked
        # filter out those that has already been assessed
        # the above can be achieved by keep only those where dist is > dist + 1 and ise
        if found_nearest_opponent
            # if found the nearest opponent then we should not find any other paths
            # we should only keep those paths that lead to another opponent
            if find_elf
                # if any of the elements is an elf then we've found it!
                iself_vec = [iself(map[new_pos]) for new_pos in new_pos]
                new_pos = new_pos[iself_vec]
            else
                # if any of the elements is an Goblin then we've found it!
                isg_vec = [isg(map[new_pos]) for new_pos in new_pos]
                new_pos = new_pos[isg_vec]
            end
        else            
            new_pos = filter(x->(path_map[x] >= dist + 1) && isunoccupied(map[x]), new_pos)            
        end
        
        # set the distance
        for np in new_pos            
            path_map[np] = dist + 1                        
        end
        
        # create the new paths to add
        new_paths_to_add = [vcat(old_paths[paths_upto], new_pos) for new_pos in new_pos]
        
        # update new_paths with the newly added paths
        for new_paths_to_add in new_paths_to_add
            push!(new_paths, new_paths_to_add)
        end
                
        #return (old_paths, path_map, found_nearest_opponent)
                
        paths_upto += 1
    end

    #writedlm("ok_$(loop).csv", path_map, ',')
    return (old_paths, path_map, found_nearest_opponent)
end

find_nearest_paths (generic function with 1 method)

In [12]:
# set up data
hw = 7

x = Array{String,2}(undef, hw, hw)
#open("15.txt") do f
#open("example1.txt") do f
open("summ2.txt") do f
    for (i,l) in enumerate(eachline(f))
        x[i,:] .= split(l,"")
    end
end
x;

map = Array{tile, 2}(undef, hw, hw)
for j=1:hw, i=1:hw
    if x[i,j] == "."
        map[i,j] = tile(true, nothing)
    elseif x[i,j] == "#"
        map[i,j] = tile(false, nothing)
    elseif x[i,j] == "G"
        map[i,j] = tile(true, ge(200, false))
    elseif x[i,j] == "E"
        map[i,j] = tile(true, ge(200, true))
    end
end
string.(map)

7×7 Array{String,2}:
 "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "E"  "."  "."  "E"  "G"  "#"
 "#"  "."  "#"  "G"  "."  "E"  "#"
 "#"  "E"  "."  "#"  "#"  "E"  "#"
 "#"  "G"  "."  "."  "#"  "."  "#"
 "#"  "."  "."  "E"  "#"  "."  "#"
 "#"  "#"  "#"  "#"  "#"  "#"  "#"

In [13]:
# a routine to find shortest path using breadth first search
pos_all = sort(findall(ischar.(map)), by=x->(x[1], x[2]));
char_up_to = 1
pos_of_char = pos_all[char_up_to]
string.(map)
# exhausted_or_enemy_path, path_map, found_nearest_opponent = find_nearest_paths(pos_of_char, map, hw)
# selected_move = sort(exhausted_or_enemy_path, by=x->(x[end][1]*33^3 +  x[end][2]*33^2 + x[2][1]*33 + x[2][2]))[1][2]        

rounds = 0 

0

In [14]:
while any(iself.(map)) && any(isg.(map))
    rounds += 1
    for pos_of_char in pos_all
        # need to check if the character has died
        if ischar(map[pos_of_char])

            # is is an Elf or Goblin
            is_elf = iself(map[pos_of_char])

            exhausted_or_enemy_path, path_map, found_nearest_opponent = find_nearest_paths(pos_of_char, map, hw)
            # make a move
            if found_nearest_opponent
                # found an enemy; ready to attack
                selected_path = sort(exhausted_or_enemy_path, by=x->(x[end][1]*33^3 +  x[end][2]*33^2 + x[2][1]*33 + x[2][2]))[1]

                # length two means that it's adjacent to an enemy
                if length(exhausted_or_enemy_path[1]) > 2                
                    map[selected_path[2]] = map[pos_of_char]
                    map[pos_of_char] = tile(true, nothing)
                end


                # able to attack
                if length(exhausted_or_enemy_path[1]) in (2,3)                
                    if length(exhausted_or_enemy_path[1]) == 2
                        curr_position = selected_path[1]
                    else 
                        curr_position = selected_path[2]
                    end                                    

                    # look in 4 directions                
                    next_pos = [curr_position] .+ CartesianIndex.([(-1, 0), (1, 0), (0, -1), (0,1)])                

                    if is_elf
                        is_next_pos_goblin = [isg(map[next_pos]) for next_pos in next_pos]
                        next_pos = next_pos[is_next_pos_goblin]
                    else
                        is_next_pos_elf = [iself(map[next_pos]) for next_pos in next_pos]
                        next_pos = next_pos[is_next_pos_elf]
                    end

                    next_pos1 = sortperm([map[next_pos].char.hp for next_pos in next_pos])[1]
                    
                    # TODO need to choose one that is earliest in reading open
                    
                    attack_pos = next_pos[next_pos1]

                    map[attack_pos] = attack(map[attack_pos])
                end

            end
            #display(string.(map))
        end

    end
    #display(string.(map))
    #println("round $rounds")
    pos_all = sort(findall(ischar.(map)), by=x->(x[1], x[2]));
end
hp = 0
for m in map
    if ischar(m)
        hp += max(0, m.char.hp)
    end
end
display(string.(map))       
rounds, hp

7×7 Array{String,2}:
 "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "."  "E"  "."  "E"  "."  "#"
 "#"  "."  "#"  "E"  "."  "."  "#"
 "#"  "E"  "."  "#"  "#"  "E"  "#"
 "#"  "."  "E"  "."  "#"  "."  "#"
 "#"  "."  "."  "."  "#"  "."  "#"
 "#"  "#"  "#"  "#"  "#"  "#"  "#"

(46, 864)

In [388]:
# sum of hp
# if none left
if !any(iself.(map)) || !any(isg.(map))
    hp = 0
    for m in map
        if ischar(m)
            hp += max(0, m.char.hp)
        end
    end
    println(hp)
end

590


In [7]:
exhausted_or_enemy_path, path_map, found_nearest_opponent = find_nearest_paths(pos_of_char, map, hw)

(Array{CartesianIndex{2},1}[[CartesianIndex(2, 2), CartesianIndex(2, 3)]], [81 81 … 81 81; 81 0 … 81 81; … ; 81 81 … 81 81; 81 81 … 81 81], true)

In [8]:
# make a move
if found_nearest_opponent
    selected_move = sort(exhausted_or_enemy_path, by=x->(x[end][1]*33^3 +  x[end][2]*33^2 + x[2][1]*33 + x[2][2]))[1][2]
end
selected_move, pos_of_char

(CartesianIndex(2, 3), CartesianIndex(2, 2))

In [247]:
map[selected_move] = map[pos_of_char]
map[pos_of_char] = tile(true, nothing)

In [250]:
char_up_to += 1

2

In [248]:
string.(map)

7×7 Array{String,2}:
 "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "."  "E"  "."  "G"  "."  "#"
 "#"  "."  "."  "."  "#"  "."  "#"
 "#"  "."  "G"  "."  "#"  "G"  "#"
 "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "#"  "#"  "#"  "#"  "#"  "#"

tile(true, nothing)

In [198]:
string.(map[1:11, 19:31])

11×13 Array{String,2}:
 "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "#"  "#"  "#"  "#"  "."  "."  "."  "#"  "#"  "#"  "#"  "#"
 "#"  "#"  "#"  "#"  "."  "."  "."  "#"  "#"  "#"  "#"  "#"  "#"
 "#"  "#"  "#"  "."  "G"  "."  "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "."  "."  "."  "G"  "."  "."  "#"  "#"  "#"  "#"  "#"  "#"  "#"
 "."  "."  "."  "."  "."  "#"  "#"  "#"  "#"  "."  "."  "#"  "#"
 "#"  "."  "G"  "."  "."  "."  "#"  "#"  "#"  "."  "."  "#"  "#"
 "."  "."  "."  "."  "G"  "."  "G"  "."  "."  "."  "."  "."  "."
 "."  "G"  "G"  "."  "."  "G"  "."  "."  "#"  "#"  "#"  "."  "."
 "."  "."  "."  "."  "."  "."  "."  "."  "."  "."  "."  "."  "#"
 "G"  "."  "."  "."  "."  "."  "."  "."  "."  "."  "."  "."  "#"

In [166]:
sorted_exhausted_or_enemy_path[1]

14-element Array{CartesianIndex{2},1}:
 CartesianIndex(4, 24) 
 CartesianIndex(4, 23) 
 CartesianIndex(5, 23) 
 CartesianIndex(6, 23) 
 CartesianIndex(6, 22) 
 CartesianIndex(7, 22) 
 CartesianIndex(8, 22) 
 CartesianIndex(9, 22) 
 CartesianIndex(10, 22)
 CartesianIndex(10, 21)
 CartesianIndex(10, 20)
 CartesianIndex(11, 20)
 CartesianIndex(12, 20)
 CartesianIndex(13, 20)

In [157]:
sorted_exhausted_or_enemy_path[2]

14-element Array{CartesianIndex{2},1}:
 CartesianIndex(4, 24) 
 CartesianIndex(4, 23) 
 CartesianIndex(5, 23) 
 CartesianIndex(6, 23) 
 CartesianIndex(6, 22) 
 CartesianIndex(7, 22) 
 CartesianIndex(8, 22) 
 CartesianIndex(9, 22) 
 CartesianIndex(10, 22)
 CartesianIndex(10, 21)
 CartesianIndex(11, 21)
 CartesianIndex(12, 21)
 CartesianIndex(13, 21)
 CartesianIndex(13, 20)

In [149]:
exhausted_or_enemy_path[2]

14-element Array{CartesianIndex{2},1}:
 CartesianIndex(4, 24) 
 CartesianIndex(4, 23) 
 CartesianIndex(5, 23) 
 CartesianIndex(6, 23) 
 CartesianIndex(6, 22) 
 CartesianIndex(7, 22) 
 CartesianIndex(8, 22) 
 CartesianIndex(9, 22) 
 CartesianIndex(10, 22)
 CartesianIndex(10, 21)
 CartesianIndex(11, 21)
 CartesianIndex(12, 21)
 CartesianIndex(13, 21)
 CartesianIndex(13, 20)

In [147]:
exhausted_or_enemy_path

2-element Array{Array{CartesianIndex{2},1},1}:
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(5, 23), CartesianIndex(6, 23), CartesianIndex(6, 22), CartesianIndex(7, 22), CartesianIndex(8, 22), CartesianIndex(9, 22), CartesianIndex(10, 22), CartesianIndex(10, 21), CartesianIndex(10, 20), CartesianIndex(11, 20), CartesianIndex(12, 20), CartesianIndex(13, 20)]
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(5, 23), CartesianIndex(6, 23), CartesianIndex(6, 22), CartesianIndex(7, 22), CartesianIndex(8, 22), CartesianIndex(9, 22), CartesianIndex(10, 22), CartesianIndex(10, 21), CartesianIndex(11, 21), CartesianIndex(12, 21), CartesianIndex(13, 21), CartesianIndex(13, 20)]

In [143]:
[(x[end], map[x[end]] |> string) for x in exhausted_or_enemy_path]
filter(x->string(map[x[end]]) == "E", exhausted_or_enemy_path)

2-element Array{Array{CartesianIndex{2},1},1}:
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(5, 23), CartesianIndex(6, 23), CartesianIndex(6, 22), CartesianIndex(7, 22), CartesianIndex(8, 22), CartesianIndex(9, 22), CartesianIndex(10, 22), CartesianIndex(10, 21), CartesianIndex(10, 20), CartesianIndex(11, 20), CartesianIndex(12, 20), CartesianIndex(13, 20)]
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(5, 23), CartesianIndex(6, 23), CartesianIndex(6, 22), CartesianIndex(7, 22), CartesianIndex(8, 22), CartesianIndex(9, 22), CartesianIndex(10, 22), CartesianIndex(10, 21), CartesianIndex(11, 21), CartesianIndex(12, 21), CartesianIndex(13, 21), CartesianIndex(13, 20)]

In [144]:
path_map[1:11, 19:31]

11×13 Array{Int64,2}:
 1024  1024  1024  1024  1024  1024  1024  1024  1024  1024  1024  1024  1024
 1024  1024  1024  1024  1024     2     3     4  1024  1024  1024  1024  1024
 1024  1024  1024  1024     2     1     2  1024  1024  1024  1024  1024  1024
 1024  1024  1024     2     1     0  1024  1024  1024  1024  1024  1024  1024
    8     7     6  1024     2     1  1024  1024  1024  1024  1024  1024  1024
    7     6     5     4     3  1024  1024  1024  1024  1024  1024  1024  1024
 1024     7  1024     5     4     5  1024  1024  1024  1024  1024  1024  1024
    9     8     7     6  1024     6  1024  1024  1024  1024  1024  1024  1024
   10  1024  1024     7     8  1024    12  1024  1024  1024  1024  1024  1024
   11    10     9     8     9    10    11    12  1024  1024  1024  1024  1024
 1024    11    10     9    10    11    12  1024  1024  1024  1024  1024  1024

In [58]:
old_paths

7-element Array{Array{CartesianIndex{2},1},1}:
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(4, 22)]
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(3, 23)]
 [CartesianIndex(4, 24), CartesianIndex(4, 23), CartesianIndex(5, 23)]
 [CartesianIndex(4, 24), CartesianIndex(3, 24), CartesianIndex(3, 23)]
 [CartesianIndex(4, 24), CartesianIndex(3, 24), CartesianIndex(2, 24)]
 [CartesianIndex(4, 24), CartesianIndex(3, 24), CartesianIndex(3, 25)]
 [CartesianIndex(4, 24), CartesianIndex(5, 24), CartesianIndex(5, 23)]

5×5 Array{Int64,2}:
 -1  -1  -1  -1  -1
 -1  -1   1  -1  -1
 -1   1   0  -1  -1
 -1  -1   1  -1  -1
 -1  -1  -1  -1  -1

In [ ]:
pos_char_char

In [ ]:
found_nearest_opponent = false

In [ ]:
display([1,3,3])

In [ ]:
path_map = Array{Int, 2}(undef, 32, 32)
path_map .= -1

In [ ]:
dist

In [ ]:
paths_upto, length(old_paths)

In [ ]:
if paths_upto > length(old_paths)
    if length(new_paths) == 0
        return "done"
    end
    old_paths = new_paths
    new_paths = Vector{CartesianIndex{2}}[]
    paths_upto = 1
    
    for op in old_paths
        path_map[op[end]] = dist
    end
    dist += 1
end

In [ ]:
old_paths, new_paths

In [ ]:
println(old_paths[paths_upto])
# obtain the last position in the path
pos = old_paths[paths_upto][end]

In [ ]:
map[pos] |> string, pos

In [ ]:
np = next_pos(pos, map, !iself(map[pos]))
dist, find_elf

In [ ]:
np

In [ ]:
# is the location is empty then set the path map to dist
for np in np
    if isempty(map[np[1]])
        path_map[np[1]] = dist
    elseif find_elf && iself(map[np[1]])
        println("woohoo found Elf")
        found_nearest_opponent = true
    elseif (!find_elf) && isg(map[np[1]])
        println("woohoo found Goblin")
        found_nearest_opponent = true
    end
end
found_nearest_opponent

In [ ]:
path_map[1:7,21:27]

In [ ]:
string.(map[2:6,22:26])

In [ ]:
new_paths_to_add = add_new_pos_to_path(old_paths[paths_upto], np)

In [ ]:
# this is the path that will be replaced
old_paths[paths_upto]

In [ ]:
new_paths

In [ ]:
new_paths_to_add[1]

In [ ]:
[push!(new_paths, new_paths_to_add) for new_paths_to_add in new_paths_to_add]
new_paths

In [ ]:
paths_upto += 1

In [ ]:
old_paths

In [ ]:
found_nearest_opponent